In [8]:
import random
import simpy
import numpy
from random import seed
import statistics
seed(29384)  # for seed of randint function
random_seed = 42  # for seed of other random generators
new_customers = 10  # Total number of customers in the system
interarrival = numpy.random.poisson(6, size=None)  # Generate new customers roughly every x seconds
waitingTimes = []
serviceTimes = []
interarrivalTimes = []

def generator(env, number, interval, server):  # customer generator with interarrival times.
    """generator generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, server, service_time=random.expovariate(0.15))
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)  # adds time to the counter, does not delete from the memory

def customer(env, name, server, service_time):
    # customer arrives to the system, waits and leaves
    arrive = env.now
    print('%7.4f : Arrival time of %s' % (arrive, name))
    with server.request() as req:
        results = yield req | env.timeout(arrive)
        
        if req in results:
            servertime = service_time
            yield env.timeout(servertime)
            serviceTimes.append(servertime)
            print('%7.4f Departure Time of %s' % (env.now, name))
            print('%7.4f Time Spent in the system of %s' % (env.now - arrive, name))
        else:
            waiting_time = env.now - arrive
            waitingTimes.append(waiting_time)
            print('%6.3f Waiting time of %s' % (waiting_time, name))

random.seed(random_seed)
env = simpy.Environment()
server = simpy.Resource(env, capacity=1)  # capacity changes the number of generators in the system.
env.process(generator(env, new_customers, interarrival, server))
env.run()
interarrivalTimes.append(interarrival)
average_interarrival = statistics.mean(interarrivalTimes)
average_waitingTime = statistics.mean(waitingTimes)
average_serviceTime = statistics.mean(serviceTimes)
print("Average Interravial Time Is : %7.4f" % (average_interarrival))
print("Average Waiting Time Is : %7.4f" % (average_waitingTime))
print("Average Service Time Is : %7.4f" % (average_serviceTime))

print("Elements of given array: ")
for i in range(0, len(interarrivalTimes)):
    print(interarrivalTimes[i]),

 0.0000 : Arrival time of Customer00
 0.1266 : Arrival time of Customer01
 0.127 Waiting time of Customer01
 1.3896 : Arrival time of Customer02
 1.390 Waiting time of Customer02
 6.8004 Departure Time of Customer00
 6.8004 Time Spent in the system of Customer00
 7.0354 : Arrival time of Customer03
 7.4902 : Arrival time of Customer04
 7.6415 : Arrival time of Customer05
11.1610 : Arrival time of Customer06
12.2695 : Arrival time of Customer07
 7.490 Waiting time of Customer04
 7.641 Waiting time of Customer05
16.2061 : Arrival time of Customer08
20.6552 : Arrival time of Customer09
21.8840 Departure Time of Customer03
14.8486 Time Spent in the system of Customer03
22.0633 Departure Time of Customer06
10.9023 Time Spent in the system of Customer06
29.0599 Departure Time of Customer07
16.7904 Time Spent in the system of Customer07
30.7201 Departure Time of Customer08
14.5140 Time Spent in the system of Customer08
41.7717 Departure Time of Customer09
21.1165 Time Spent in the system of C